In [ ]:
%cd ../input/icevision-080

In [ ]:
!pip install timm-0.4.9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install effdet-0.2.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install omegaconf-2.0.6-py3-none-any.whl -f ./ --no-index --no-deps

In [ ]:
%cd ..

In [ ]:
%cd pycocotools202/

In [ ]:
!pip install pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl -f ./ --no-index --no-deps

In [ ]:
%cd ..

In [ ]:
%cd pydicom-helper

In [ ]:
!conda install 'libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install 'libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install 'gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install 'conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install 'certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install 'openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
%cd ..

In [ ]:
!pip install ../input/icevision08-kagcuda/fastai-2.3.1-py3-none-any.whl --no-deps -q

# Imports

In [ ]:
import pandas as pd
import numpy as np
import re
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import albumentations as A
from tqdm.notebook import tqdm
import gc

import timm
from fastai.vision.all import *
from fastai.medical.imaging import *
from fastai.vision.learner import create_head
from fastai.tabular.all import *

In [ ]:
from effdet import get_efficientdet_config, create_model_from_config, unwrap_bench, create_model, load_checkpoint
from effdet.bench import _post_process, _batch_detection
from effdet.config import set_config_readonly, set_config_writeable
from effdet.efficientdet import get_feature_info, BiFpn, BiFpnLayer, HeadNet, _init_weight, _init_weight_alt
from effdet.loss import DetectionLoss
from effdet.anchors import Anchors, AnchorLabeler

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Preparing Data

In [ ]:
sz = 512
bs = 32
mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225]

In [ ]:
box_class = {'background': 0, 'opacity': 1, 'none': 2}
classi_class = {'Negative for Pneumonia': 0, 'Typical Appearance': 1, 'Indeterminate Appearance': 2, 'Atypical Appearance': 3}

In [ ]:
cc_i2o = {classi_class[k]:k for k in classi_class.keys()}

In [ ]:
bc_i2o = {box_class[k]:k for k in box_class.keys()}

In [ ]:
path_tst_img = Path('../input/siim-covid19-detection/test'); path_tst_img.ls()

In [ ]:
pairs = []
for stu in path_tst_img.ls():
    study = str(stu).split('/')[-1]
    for images in stu.ls(): 
        for im in images.ls():
            pairs.append([study, str(im).split('/')[-1], im])

In [ ]:
df = pd.DataFrame(pairs,columns=['Study', 'Image', 'fname'])

In [ ]:
df['Ori_x'] = None
df['Ori_y'] = None
df['PatientSex'] = None
df['Modality'] = None
for i, fn in tqdm(enumerate(df['fname'].values)):
    dcm = Path(fn).dcmread()
    y = dcm.Rows
    x = dcm.Columns
    ps = dcm.PatientSex
    mod = dcm.Modality
    df.loc[i, 'Ori_x'] = x
    df.loc[i, 'Ori_y'] = y
    df.loc[i, 'PatientSex'] = ps
    df.loc[i, 'Modality'] = mod

In [ ]:
df['x_scale'] = df['Ori_x']/sz
df['y_scale'] = df['Ori_y']/sz

In [ ]:
df['Modality'] = df['Modality'].map({'DX':0, 'CR':1})
df['PatientSex'] = df['PatientSex'].map({'M':0, 'F':1})

In [ ]:
ss = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [ ]:
len(ss) == (len(path_tst_img.ls()) + len(df))

In [ ]:
'''
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
'''

In [ ]:
 def dicom2array(path, voi_lut=True, fix_monochrome=True):
        dicom = pydicom.read_file(path)
        
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        
        if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.amax(data) - data
        
        data = data - np.min(data)
        data = data / np.max(data)
        data = (data * 255).astype(np.uint8)
        
        #image = self.transform(date)
        
        return data

In [ ]:
class CDataset(Dataset):
    def __init__(self, df, tfms=None):
        self.df = df
        self.tfms = tfms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        
        img_path  = self.df.loc[index, 'fname']
        ps = self.df.loc[index, 'PatientSex']
        mod = self.df.loc[index, 'Modality']
        img = dicom2array(img_path)
        img = PILImage.create(img)
        tfm_img = self.tfms(img)
        
        return torch.cat([tfm_img, tfm_img, tfm_img]), torch.Tensor([mod, ps]).type(torch.LongTensor)

In [ ]:
tfms = transforms.Compose([transforms.Resize((sz, sz)), 
                           transforms.ToTensor(), 
                           transforms.Normalize(mean=[0.485], std=[0.229])])

In [ ]:
dataset = CDataset(df, tfms=tfms)

In [ ]:
loader = DataLoader(dataset=dataset, batch_size=bs, shuffle=False)

In [ ]:
x,y= next(iter(loader))

# Preparing and loading model/bench

In [ ]:
#modeified effdet's EfficientDet to calculate classification output from backbone's output
class EfficientDetClassi(nn.Module):
    def __init__(self, config, pretrained_backbone=True, classi_class=None, alternate_init=False):
        super(EfficientDetClassi, self).__init__()
        self.config = config
        set_config_readonly(self.config)
        self.backbone = timm.create_model(
            config.backbone_name, features_only=True,
            out_indices=self.config.backbone_indices or (2, 3, 4),
            pretrained=pretrained_backbone, **config.backbone_args)
        feature_info = get_feature_info(self.backbone)
        self.fpn = BiFpn(self.config, feature_info)
        self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
        self.box_net = HeadNet(self.config, num_outputs=4)

        #INSERTED CODE STARTS
        backbone_features = num_features_model(nn.Sequential(*self.backbone.children()))
        self.vis_head = create_head(backbone_features, 100) #fastai's create_head
        self.tab = TabularModel(emb_szs, n_cont=0, out_sz = 100, layers = [100, 250])
        self.final_head = nn.Sequential(
                                        nn.BatchNorm1d(200),
                                        nn.Dropout(0.25),
                                        nn.Linear(200, 100, bias=False),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(100),
                                        nn.Dropout(0.5),
                                        nn.Linear(100, 4)
                                            )  
        #INSERTED CODE ENDS
        '''
        self.classifier = nn.Sequential(
                                        nn.AdaptiveMaxPool2d(output_size=1),
                                        nn.Flatten(),
                                        nn.BatchNorm1d(backbone_features),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(backbone_features, 512),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(512),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(512, classi_class), 
        )
  
        '''
        for n, m in self.named_modules():
            if 'backbone' not in n:
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def reset_head(self, num_classes=None, aspect_ratios=None, num_scales=None, alternate_init=False):
        reset_class_head = False
        reset_box_head = False
        set_config_writeable(self.config)
        if num_classes is not None:
            reset_class_head = True
            self.config.num_classes = num_classes
        if aspect_ratios is not None:
            reset_box_head = True
            self.config.aspect_ratios = aspect_ratios
        if num_scales is not None:
            reset_box_head = True
            self.config.num_scales = num_scales
        set_config_readonly(self.config)

        if reset_class_head:
            self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
            for n, m in self.class_net.named_modules(prefix='class_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

        if reset_box_head:
            self.box_net = HeadNet(self.config, num_outputs=4)
            for n, m in self.box_net.named_modules(prefix='box_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def toggle_head_bn_level_first(self):
        """ Toggle the head batchnorm layers between being access with feature_level first vs repeat
        """
        self.class_net.toggle_bn_level_first()
        self.box_net.toggle_bn_level_first()

    def forward(self, x, tab):
        x_b = self.backbone(x)
        x = self.fpn(x_b)
        x_class = self.class_net(x)
        x_box = self.box_net(x)
        x_classi_vis = self.vis_head(x_b[2])   #INSERTED CODE
        x_class_tab = self.tab(tab)            #INSERTED CODE
        x_classi = self.final_head(torch.cat([x_class_tab, x_classi_vis], dim=1))
        return x_class, x_box, x_classi #returns x_classi on top of original

In [ ]:
class DetClassiBenchPredict(nn.Module):
    def __init__(self, model):
        super(DetClassiBenchPredict, self).__init__()
        self.model = model
        self.config = model.config  # FIXME remove this when we can use @property (torchscript limitation)
        self.num_levels = model.config.num_levels
        self.num_classes = model.config.num_classes
        self.anchors = Anchors.from_config(model.config)
        self.max_detection_points = model.config.max_detection_points
        self.max_det_per_image = model.config.max_det_per_image
        self.soft_nms = model.config.soft_nms

    def forward(self, x, tab, img_info=None):
        class_out, box_out, classi_out = self.model(x, tab) #OUTPUTS INCLUDE classiicaiton predictions
        
        class_out, box_out, indices, classes = _post_process(
            class_out, box_out, num_levels=self.num_levels, num_classes=self.num_classes,
            max_detection_points=self.max_detection_points)
        if img_info is None:
            img_scale, img_size = None, None
        else:
            img_scale, img_size = img_info['img_scale'], img_info['img_size']
        return _batch_detection(
            x.shape[0], class_out, box_out, self.anchors.boxes, indices, classes,
            img_scale, img_size, max_det_per_image=self.max_det_per_image, soft_nms=self.soft_nms
        ), classi_out


In [ ]:
def create_model_m(model_name, bench_task='', num_classes=None, pretrained=False,
                 checkpoint_path='', checkpoint_ema=False, img_size=None, **kwargs):

    config = get_efficientdet_config(model_name)
    config.image_size = (img_size, img_size) if isinstance(img_size, int) else img_size

    return create_model_from_config_m(config, bench_task=bench_task, num_classes=num_classes, pretrained=pretrained,
                                      checkpoint_path=checkpoint_path, checkpoint_ema=checkpoint_ema, **kwargs)

In [ ]:
def create_model_from_config_m(
        config, bench_task='', num_classes=None, pretrained=False,
        checkpoint_path='', checkpoint_ema=False, **kwargs):

    pretrained_backbone = kwargs.pop('pretrained_backbone', False)
    if pretrained or checkpoint_path:
        pretrained_backbone = False  # no point in loading backbone weights

    # Config overrides, override some config values via kwargs.
    overrides = (
        'redundant_bias', 'label_smoothing', 'legacy_focal', 'jit_loss', 'soft_nms', 'max_det_per_image', 'image_size')
    for ov in overrides:
        value = kwargs.pop(ov, None)
        if value is not None:
            setattr(config, ov, value)

    labeler = kwargs.pop('bench_labeler', False)

    # create the base model
    model = EfficientDetClassi(config, pretrained_backbone=pretrained_backbone, **kwargs)
    
    # pretrained weights are always spec'd for original config, load them before we change the model
    if pretrained:
        load_pretrained(model, config.url)

    # reset model head if num_classes doesn't match configs
    if num_classes is not None and num_classes != config.num_classes:
        model.reset_head(num_classes=num_classes)

    # load an argument specified training checkpoint
    if checkpoint_path:
        load_checkpoint(model, checkpoint_path, use_ema=checkpoint_ema)

    # wrap model in task specific training/prediction bench if set
    if bench_task == 'train':
        model = DetClassiBenchTrain(model, create_labeler=True)
    elif bench_task == 'predict':
        model = DetClassiBenchPredict(model)
    return model

In [ ]:
def splitter(m):
    s = nn.Sequential(*m.model.children())
    return L(s[0], s[1:]).map(params)

In [ ]:
emb_szs = [(3, 3), (3, 3)]

In [ ]:
bench = create_model_m('tf_efficientdet_d3_ap', 
                        'predict',  
                        num_classes=2,
                        classi_class=4,
                        img_size=sz)

# Inference

In [ ]:
preds = []
classis = []

for i in range(3):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    PATH = f'../input/detclassitab-d3-ap-10/tf_efficientdet_d3_ap_{i}.pth'
    bench.load_state_dict(torch.load(PATH))
    bench.to(device)
    bench.model.eval()
    
    pred = None
    classi = None
    for batch_idx, data in tqdm(enumerate(loader), total=len(loader)):
        img, tab = data[0].to(device=device), data[1].to(device=device)
        with torch.no_grad():
            p, c = bench(img, tab)
            if pred == None:
                pred = p.detach().cpu()
                classi = c.detach().cpu()
            else:
                pred = torch.cat([pred, p.detach().cpu()], 0)
                classi = torch.cat([classi, c.detach().cpu()], 0)
        gc.collect()
    preds.append(pred)
    classis.append(classi)

In [ ]:
sm = nn.Softmax(dim=-1)
classi_preds = (sm(classis[0]).numpy() + sm(classis[1]).numpy() + sm(classis[2]).numpy())/3

In [ ]:
preds = torch.cat([preds[0][:,:5,:], preds[1][:,:5,:], preds[2][:,:5,:]], 1).numpy()

# Preparing submission file

In [ ]:
df['classification_cls']  = np.argmax(classi_preds,1)
df['classification_cls'] = df['classification_cls'].map(cc_i2o)
df['classification_preds'] = df.apply(lambda row: classi_preds[row.name], axis=1)

In [ ]:
def prediction_stu(row):
    stu = row.Study
    sel_df = df[df['Study'] == stu]
    p = sum(sel_df.classification_preds)/len(sel_df)
    predstr = f'negative {p[0]} 0 0 1 1 typical {p[1]} 0 0 1 1 intermediate {p[2]} 0 0 1 1 atypical {p[3]} 0 0 1 1' 
    return predstr

In [ ]:
df['PredStrStu'] = df.apply(lambda row: prediction_stu(row), 1)
stu_ = []
for stu in df['Study'].unique():
    stu_.append([f'{stu}_study', df[df['Study']==stu]['PredStrStu'].iloc[0]])
df_stu = pd.DataFrame(stu_, columns=['id', 'PredictionString'])

In [ ]:
df['bboxes'] = list(preds)

In [ ]:
def scale_bboxes(row):
    x_s = row.x_scale
    y_s = row.x_scale
    scale = [x_s, y_s, x_s, y_s]
    
    rescaled_boxes = []
    for box in row.bboxes:
        b = list(box[:4]*scale)
        b.append(box[4])
        b.append(box[5])
        
        rescaled_boxes.append(b)
    return rescaled_boxes

In [ ]:
df['rescaled_bboxes'] = df.apply(lambda x: scale_bboxes(x), axis=1)

In [ ]:
def prediction_img(row, thres=0.35):
    
    prediction_string = ""
    
    if row.classification_cls == 'Negative for Pneumonia':
        prediction_string = f'none {row.classification_preds[0]} 0 0 1 1'
    
    elif row.classification_cls != 'Negative for Pneumonia':
        for box in row.rescaled_bboxes:
            if box[4] > thres:
                if box[5] == 1.:
                    if prediction_string == "":
                        prediction_string = f'opacity {box[4]} {box[0]} {box[1]} {box[2]} {box[3]}'
                    else:
                        prediction_string += f' opacity {box[4]} {box[0]} {box[1]} {box[2]} {box[3]}'
    
    return prediction_string

In [ ]:
df['PredStrImg'] = df.apply(lambda row: prediction_img(row), 1)

In [ ]:
df_img = df[['Image', 'PredStrImg']].copy()
df_img['id'] = df_img['Image'].apply(lambda x: f'{x.split(".dcm")[0]}_image')
df_img.drop('Image', axis=1, inplace=True)
df_img.columns = ['PredictionString', 'id']
df_img = df_img[['id', 'PredictionString']]

In [ ]:
df_sub = pd.concat([df_stu, df_img])

In [ ]:
df_sub = df_sub[['id', 'PredictionString']].reset_index(drop=True)

In [ ]:
df_sub.to_csv('/kaggle/working/submission.csv', index=False)

# Analysing Predictions 

In [ ]:
def prepare_bboxes(preds, x_scale, y_scale):
    bboxes = []
    bconfs = []
    bclses = []
    for pred in preds:
        bbox = pred[:4]
        bbox = bbox * [x_scale, y_scale, x_scale, y_scale]
        bboxes.append(bbox)
        
        bconfs.append(pred[4])
        bclses.append(pred[5])
        
    return bboxes, bconfs, bclses 

In [ ]:
def draw_bbox(img, bboxes, bconfs, bclses, color=(255, 0, 0), thickness=5):
    
    img1 = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)    

    for i, bbox in enumerate(bboxes):
        x_min, y_min, x_max, y_max = map(int, bbox)
        cv2.rectangle(img1, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
        cv2.putText(img1, bc_i2o[bclses[i]], (x_min, y_min-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, 
                    thickness=thickness, color=(255,255,255))
        cv2.putText(img1, str(f'{bconfs[i]:.4f}'), (x_min, y_min-120), 
                    cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, 
                    thickness=thickness, color=(255,255,255))
        '''
        if target is not None:
            y_min, x_min, y_max, x_max = map(int, target)
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=(0,255,0), thickness=thickness)
        '''
    return img1

In [ ]:
def show_prediction(index, preds, classi, vocab, sm=nn.Softmax()):
    
    preds = preds[index]
    preds = np.array(preds.detach().cpu())
    classi = classi[index].detach().cpu()
    
    classi = np.argmax(np.array(sm(classi)))
    predicted_label = vocab[classi]
    
    fn = df.fname[index]
    x_scale = df.x_scale[index]
    y_scale = df.y_scale[index]
    
    img = dicom2array(fn)
    bboxes, bconfs, bclses = prepare_bboxes(preds, x_scale, y_scale)
    img_bboxes = draw_bbox(img, bboxes, bconfs, bclses)
    
    plt.figure(figsize=(12,10))
    plt.imshow(img_bboxes, cmap='gray')
    plt.title(predicted_label)
    
    #return img_bboxes